In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logesstic_example').getOrCreate()

24/08/07 14:55:02 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/07 14:55:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 14:55:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/07 14:55:03 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/07 14:55:03 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/07 14:55:03 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/08/07 14:55:03 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
df = spark.read.csv('titanic.csv', inferSchema= True, header = True)

In [4]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [6]:
my_cols = df.select(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','Embarked'])

## Deal with missing data

In [8]:
my_final_data = my_cols.na.drop() # here for the simplicity we drop the missing data

In [9]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [10]:
gender_indexer = StringIndexer(inputCol = 'Sex', outputCol = 'SexIndex')

# first convert strings to numbers then convert numbers to binary values
## StringIndexer:
## A B C --> 0 1 2
## One hot encoder:
## key A B C
## Example A --> [1, 0, 0] no B and C
## Example B --> [0, 1, 0]
## Example C --> [0, 0, 1]

In [16]:
gender_encoder = OneHotEncoder(inputCol = 'SexIndex', outputCol = 'SexVec')

In [17]:
embark_indexer = StringIndexer(inputCol = 'Embarked', outputCol = 'EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol = 'EmbarkIndex', outputCol = 'EmbarkVec')

In [18]:
assembler = VectorAssembler(inputCols = ['Pclass', 'SexVec', 'EmbarkVec', 'Age', 'SibSp', 'Parch', 'Fare'],
                            outputCol = 'features')

In [19]:
from pyspark.ml.classification import LogisticRegression

## import pipeline to set stages for different steps

In [20]:
from pyspark.ml import Pipeline

In [21]:
log_reg_titanic = LogisticRegression(featuresCol = 'features', labelCol = 'Survived')

In [27]:
pipeline = Pipeline(stages = [gender_indexer, embark_indexer, gender_encoder, 
                               embark_encoder, assembler, log_reg_titanic])

In [28]:
train_data, test_data = my_final_data.randomSplit([0.7, 0.3])

In [31]:
fit_model = pipeline.fit(train_data)

In [32]:
results = fit_model.transform(test_data)

In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [34]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction', labelCol = 'Survived')

In [35]:
results.select('Survived', 'prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [36]:
AUC = my_eval.evaluate(results)

AUC